In [1]:
import pyarrow
import pymapd
import pandas as pd
import geopandas as gpd 
import shapely
from shapely.geometry import Point, LineString, Polygon, MultiPolygon 

In [2]:
def cursor2df(cursor):
    has_geodata = {
        c.name: c.type_code in [13, 14, 15] for c in cursor.description
    }
    col_names = [c.name for c in cursor.description]

    df_class = gpd.GeoDataFrame if any(has_geodata.values()) else pd.DataFrame
    df = df_class(cursor.fetchall(), columns=col_names)

    for c, _has_geodata in has_geodata.items():
        if _has_geodata:
            df.loc[:, c] = df.loc[:, c].apply(shapely.wkt.loads)

    return df


In [3]:
con = pymapd.connect(
    host='localhost', user='admin', password='HyperInteractive',
    protocol='http', port='6278'
)

In [4]:
con._client.get_version()

'5.2.0dev-20200329-cecceef8da'

## Geospatial fields

### test 1

In [26]:
table_name = 'mytablegeo'
con.execute('''
DROP TABLE IF EXISTS {};
'''.format(table_name))

con.execute('''
CREATE TABLE {} (
    a POINT, b LINESTRING, c POLYGON
);
'''.format(table_name))

In [27]:
df =  gpd.GeoDataFrame(
    {
        'a': [
            shapely.geometry.Point(0, 0),
            shapely.geometry.Point(1, 1),
        ],
        'b': [
            shapely.geometry.LineString(
                [(2, 0), (2, 4), (3, 4)]
            ),
            shapely.geometry.LineString([(0, 0), (1, 1)]),
        ],
        'c': [
            shapely.geometry.Polygon(
                [(0, 0), (1, 0), (0, 1), (0, 0)]
            ),
            shapely.geometry.Polygon(
                [[0, 0], [4, 0], [4, 4], [0, 4], [0, 0]]
            ),
        ],
    }
)
df

,a,b,c
0,POINT (0 0),"LINESTRING (2 0, 2 4, 3 4)","POLYGON ((0 0, 1 0, 0 1, 0 0))"
1,POINT (1 1),"LINESTRING (0 0, 1 1)","POLYGON ((0 0, 4 0, 4 4, 0 4, 0 0))"


In [28]:
con.load_table(table_name, df, method='columnar')

cur = con.execute('select * from {}'.format(table_name))

df = cursor2df(cur)

df

,a,b,c
0,POINT (0 0),"LINESTRING (2 0, 2 4, 3 4)","POLYGON ((0 0, 1 0, 0 1, 0 0))"
1,POINT (1 1),"LINESTRING (0 0, 1 1)","POLYGON ((0 0, 4 0, 4 4, 0 4, 0 0))"


### test 2

In [29]:
table_name = 'mytablegeo'
con.execute('''
DROP TABLE IF EXISTS {};
'''.format(table_name))

con.execute('''
CREATE TABLE {} (
    a POINT, 
    b LINESTRING, 
    c POLYGON, 
    d MULTIPOLYGON, 
    e INT,
    f TEXT,
    g DOUBLE
);
'''.format(table_name))

In [30]:
df =  gpd.GeoDataFrame(
    {
        'a': [Point(0, 0), Point(1, 1)],
        'b': [
            LineString([(2, 0), (2, 4), (3, 4)]),
            LineString([(0, 0), (1, 1)]),
        ],
        'c': [
            Polygon([(0, 0), (1, 0), (0, 1), (0, 0)]),
            Polygon([[0, 0], [4, 0], [4, 4], [0, 4], [0, 0]]),
        ],
        'd': [
            MultiPolygon(
                [
                    Polygon([(0, 0), (1, 1), (1, 0), (0, 0)]),
                    Polygon(
                        [
                            (0, 0),
                            (4, 0),
                            (4, 4),
                            (0, 4),
                            (0, 0),
                        ]
                    ),
                ]
            ),
            MultiPolygon(
                [
                    Polygon(
                        [
                            (0, 0),
                            (4, 0),
                            (4, 4),
                            (0, 4),
                            (0, 0),
                        ]
                    ),
                    Polygon([(0, 0), (1, 1), (1, 0), (0, 0)]),
                ]
            ),
        ],
        'e': [1, 2],
        'f': ['A', 'B'],
        'g': [1.0, 2.2],
    }
)
df

,a,b,c,d,e,f,g
0,POINT (0 0),"LINESTRING (2 0, 2 4, 3 4)","POLYGON ((0 0, 1 0, 0 1, 0 0))","(POLYGON ((0 0, 1 1, 1 0, 0 0)), POLYGON ((0 0...",1,A,1.0
1,POINT (1 1),"LINESTRING (0 0, 1 1)","POLYGON ((0 0, 4 0, 4 4, 0 4, 0 0))","(POLYGON ((0 0, 4 0, 4 4, 0 4, 0 0)), POLYGON ...",2,B,2.2


In [31]:
con.load_table(table_name, df, method='columnar')

cur = con.execute('select * from {}'.format(table_name))

df = cursor2df(cur)

df

,a,b,c,d,e,f,g
0,POINT (0 0),"LINESTRING (2 0, 2 4, 3 4)","POLYGON ((0 0, 1 0, 0 1, 0 0))","MULTIPOLYGON (((0 0,1 0,1 1,0 0)),((0 0,4 0,4 ...",1,A,1.0
1,POINT (1 1),"LINESTRING (0 0, 1 1)","POLYGON ((0 0, 4 0, 4 4, 0 4, 0 0))","MULTIPOLYGON (((0 0,4 0,4 4,0 4,0 0)),((0 0,1 ...",2,B,2.2


## Auto create table

In [32]:
table_name = 'myautotableageo'
con.execute('''
DROP TABLE IF EXISTS {};
'''.format(table_name))

In [33]:
df =  gpd.GeoDataFrame(
    {
        'a': [Point(0, 0), Point(1, 1)],
        'b': [
            LineString([(2, 0), (2, 4), (3, 4)]),
            LineString([(0, 0), (1, 1)]),
        ],
        'c': [
            Polygon([(0, 0), (1, 0), (0, 1), (0, 0)]),
            Polygon([[0, 0], [4, 0], [4, 4], [0, 4], [0, 0]]),
        ],
        'd': [
            MultiPolygon(
                [
                    Polygon([(0, 0), (1, 1), (1, 0), (0, 0)]),
                    Polygon(
                        [
                            (0, 0),
                            (4, 0),
                            (4, 4),
                            (0, 4),
                            (0, 0),
                        ]
                    ),
                ]
            ),
            MultiPolygon(
                [
                    Polygon(
                        [
                            (0, 0),
                            (4, 0),
                            (4, 4),
                            (0, 4),
                            (0, 0),
                        ]
                    ),
                    Polygon([(0, 0), (1, 1), (1, 0), (0, 0)]),
                ]
            ),
        ],
        'e': [1, 2],
        'f': ['A', 'B'],
        'g': [1.0, 2.2],
    }
)
df

,a,b,c,d,e,f,g
0,POINT (0 0),"LINESTRING (2 0, 2 4, 3 4)","POLYGON ((0 0, 1 0, 0 1, 0 0))","(POLYGON ((0 0, 1 1, 1 0, 0 0)), POLYGON ((0 0...",1,A,1.0
1,POINT (1 1),"LINESTRING (0 0, 1 1)","POLYGON ((0 0, 4 0, 4 4, 0 4, 0 0))","(POLYGON ((0 0, 4 0, 4 4, 0 4, 0 0)), POLYGON ...",2,B,2.2


In [34]:
con.load_table(table_name, df, method='columnar')

cur = con.execute('select * from {}'.format(table_name))

df = cursor2df(cur)

df

> /home/xmn/dev/quansight/omnisci-project/pymapd/pymapd/_pandas_loaders.py(285)build_row_desc()
-> return row_desc


(Pdb)  p row_desc


[TColumnType(col_name='a', col_type=TTypeInfo(type=13, encoding=None, nullable=None, is_array=None, precision=23, scale=None, comp_param=None, size=-1), is_reserved_keyword=None, src_name=None, is_system=None, is_physical=None, col_id=None), TColumnType(col_name='b', col_type=TTypeInfo(type=14, encoding=None, nullable=None, is_array=None, precision=23, scale=None, comp_param=None, size=-1), is_reserved_keyword=None, src_name=None, is_system=None, is_physical=None, col_id=None), TColumnType(col_name='c', col_type=TTypeInfo(type=15, encoding=None, nullable=None, is_array=None, precision=23, scale=None, comp_param=None, size=-1), is_reserved_keyword=None, src_name=None, is_system=None, is_physical=None, col_id=None), TColumnType(col_name='d', col_type=TTypeInfo(type=16, encoding=None, nullable=None, is_array=None, precision=23, scale=None, comp_param=None, size=-1), is_reserved_keyword=None, src_name=None, is_system=None, is_physical=None, col_id=None), TColumnType(col_name='e', col_type=

(Pdb)  c


,a,b,c,d,e,f,g
0,POINT (0 0),"LINESTRING (2 0, 2 4, 3 4)","POLYGON ((0 0, 1 0, 0 1, 0 0))","MULTIPOLYGON (((0 0,1 0,1 1,0 0)),((0 0,4 0,4 ...",1,A,1.0
1,POINT (1 1),"LINESTRING (0 0, 1 1)","POLYGON ((0 0, 4 0, 4 4, 0 4, 0 0))","MULTIPOLYGON (((0 0,4 0,4 4,0 4,0 0)),((0 0,1 ...",2,B,2.2


In [35]:
table_name = 'myautotableageo'
con.execute('''
DROP TABLE IF EXISTS {};
'''.format(table_name))

cur = con.execute('CREATE TABLE {} ( a GEOMETRY(MULTIPOLYGON))'.format(table_name))

con.get_table_details(table_name)

[ColumnDetails(name='a', type='MULTIPOLYGON', nullable=True, precision=23, scale=0, comp_param=0, encoding='NONE', is_array=False)]